In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
file = 'SAV/data/All_SAV_Parameters-2023-Jun-05.txt'
SAV = pd.read_csv(file, sep='|',na_values=['NULL','','NA'])

C:\Users\Hill_T\AppData\Local\Temp\ipykernel_1680\1126649852.py:2: DtypeWarning: Columns (17,23) have mixed types. Specify dtype option on import or set low_memory=False.
  SAV = pd.read_csv(file, sep='|',na_values=['NULL','','NA'])


In [6]:
SAV['BB'] = SAV[SAV.ParameterName=='Braun Blanquet Score']['ResultValue']
SAV['mBB'] = SAV[SAV.ParameterName=='Modified Braun Blanquet Score']['ResultValue']
SAV['PC'] = SAV[SAV.ParameterName=='Percent Cover']['ResultValue']
SAV['PO'] = SAV[SAV.ParameterName=='Percent Occurrence']['ResultValue']
SAV['SC'] = SAV[SAV.ParameterName=='Shoot Count']['ResultValue']
SAV['PA'] = SAV[SAV.ParameterName=='Presence/Absence']['ResultValue']

In [7]:
SAV[SAV.CommonIdentifier=='Total_SAV'].CommonIdentifier = 'Total SAV'
SAV_sum = SAV.groupby('ManagedAreaName').agg(n_yr=('Year', lambda x: len(np.unique(x))), yrs=('Year', lambda x: sorted(np.unique(x).tolist())))

C:\Users\Hill_T\AppData\Local\Temp\ipykernel_1680\1331889215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SAV[SAV.CommonIdentifier=='Total_SAV'].CommonIdentifier = 'Total SAV'


In [8]:
SAV2 = SAV[(~pd.isna(SAV['BB'])) | (~pd.isna(SAV['mBB'])) | (~pd.isna(SAV['PC'])) | (~pd.isna(SAV['PO']))]

SAV2 = SAV2[(SAV2['BB'] >= 0) & (SAV2['BB'] <= 5) | pd.isna(SAV2['BB'])]
SAV2 = SAV2[(SAV2['mBB'] >= 0) & (SAV2['mBB'] <= 5) | pd.isna(SAV2['mBB'])]
SAV2 = SAV2[(SAV2['PC'] >= 0) & (SAV2['PC'] <= 100) | pd.isna(SAV2['PC'])]
SAV2 = SAV2[(SAV2['PO'] >= 0) & (SAV2['PO'] <= 100) | pd.isna(SAV2['PO'])]
SAV2 = SAV2[SAV2['Month'].isin(list(range(4, 11)))]

In [9]:
SAV2['BB_all'] = np.nan

In [10]:
SAV2.loc[~SAV2['BB'].isna(),'BB_all'] = np.select(
    [(SAV2[~SAV2['BB'].isna()]['BB'] == 0),(SAV2[~SAV2['BB'].isna()]['BB'] > 0) & (SAV2[~SAV2['BB'].isna()]['BB'] <= 1),(SAV2[~SAV2['BB'].isna()]['BB'] > 1)], 
    [0, 1, np.round(SAV2[~SAV2['BB'].isna()]['BB'])]
)

SAV2.loc[~SAV2['mBB'].isna(), 'BB_all'] = np.select(
    [(SAV2[~SAV2['mBB'].isna()]['mBB']==0),
        (SAV2[~SAV2['mBB'].isna()]['mBB'] > 0) & (SAV2[~SAV2['mBB'].isna()]['mBB'] <= 1),
        (SAV2[~SAV2['mBB'].isna()]['mBB'] > 1)],
    [0, 1, np.round(SAV2[~SAV2['mBB'].isna()]['mBB'])]
)

pc_na = SAV2.loc[~SAV2['PC'].isna()]['PC']
pc_conditions = [
    (pc_na==0),
    ((pc_na > 0)&(pc_na <= (2.5+(15-2.5)/2))),
    (pc_na <= (2.5+(15-2.5)+(37.5-15)/2)),
    (pc_na <= (2.5+(15-2.5)+(37.5-15)+(62.5-37.5)/2)),
    (pc_na <= (2.5+(15-2.5)+(37.5-15)+(62.5-37.5)+(87.5-62.5)/2)),
    (pc_na > (2.5+(15-2.5)+(37.5-15)+(62.5-37.5)+(87.5-62.5)/2))
]
SAV2.loc[~SAV2['PC'].isna(), 'BB_all'] = np.select(pc_conditions,[0, 1, 2, 3, 4, 5])

In [11]:
bb_all_na = SAV2.loc[~SAV2['BB_all'].isna()]
SAV2.loc[~SAV2['BB_all'].isna(), 'BB_pct'] = np.select(
    [
        bb_all_na['BB_all'] == 0,
        (bb_all_na['BB_all'] > 0) & (bb_all_na['BB_all'] <= 0.1),
        (bb_all_na['BB_all'] > 0.1) & (bb_all_na['BB_all'] <= 0.5),
        (bb_all_na['BB_all'] > 0.5) & (bb_all_na['BB_all'] <= 1),
        (bb_all_na['BB_all'] > 1) & (bb_all_na['BB_all'] <= 2),
        (bb_all_na['BB_all'] > 2) & (bb_all_na['BB_all'] <= 3),
        (bb_all_na['BB_all'] > 3) & (bb_all_na['BB_all'] <= 4),
        (bb_all_na['BB_all'] > 4) & (bb_all_na['BB_all'] <= 5)
    ],
    [
        0,
        np.interp(bb_all_na['BB_all'], [0, 0.1], [0, 0.02]),
        np.interp(bb_all_na['BB_all'], [0.1, 0.5], [0.02, 0.1]),
        np.interp(bb_all_na['BB_all'], [0.5, 1], [0.1, 2.5]),
        np.interp(bb_all_na['BB_all'], [1, 2], [2.5, 15]),
        np.interp(bb_all_na['BB_all'], [2, 3], [15, 37.5]),
        np.interp(bb_all_na['BB_all'], [3, 4], [37.5, 62.5]),
        np.interp(bb_all_na['BB_all'], [4, 5], [62.5, 87.5])
    ],
    default=np.nan
)

SAV2['BB_pct'] = SAV2['BB_pct'].astype(float)
SAV2['BB_all'] = pd.Categorical(SAV2['BB_all'])

SAV2.loc[~SAV2['PO'].isna(), 'method'] = "Percent Occurrence"
SAV2.loc[~SAV2['BB'].isna(), 'method'] = "Braun Blanquet"
SAV2.loc[~SAV2['mBB'].isna(), 'method'] = "Modified Braun Blanquet"
SAV2.loc[~SAV2['PC'].isna(), 'method'] = "Percent Cover"

In [12]:
import re

# Update PA where BB_all is not NA
SAV2.loc[~SAV2['BB_all'].isna(), 'PA'] = np.where(SAV2.loc[~SAV2['BB_all'].isna()]['BB_all'] == 0, 0, 1)

# Update PA where PO is not NA
SAV2.loc[~SAV2['PO'].isna(), 'PA'] = np.where(SAV2.loc[~SAV2['PO'].isna()]['PO'] == 0, 0, 1)

# Calculate relyear
SAV2['relyear'] = SAV2['Year'] - SAV2['Year'].min()

In [13]:
# Filter SAV2 for SpeciesGroup1 values "Seagrass" or "Macroalgae"
SAV3 = SAV2[SAV2['SpeciesGroup1'].isin(['Seagrass', 'Macroalgae'])].copy()

In [14]:
# Temporary fix for ProgramID values 570 and 571
SAV3.loc[(SAV3['ProgramID'].isin([570, 571])) & (SAV3['CommonIdentifier'] == 'Total SAV'), 'CommonIdentifier'] = 'Total seagrass'

# Temporary fix for records with CommonIdentifier as "Drift algae"
SAV3.loc[SAV3['CommonIdentifier'] == 'Drift algae', 'Drift_Attached'] = 'Drift'

In [15]:
species_reject = ["All", "NA",
                  "Vallisneria americana", "Najas guadalupensis",
                  "Hydrilla verticillata", "Potamogeton pusillus",
                  "Zannichellia palustris"]

# Update analysisunit_halid and analysisunit columns
SAV3['analysisunit_halid'] = np.where(SAV3['CommonIdentifier'].isin(species_reject), np.nan,
                                      np.where(SAV3['CommonIdentifier'].str.contains('Halophila') & SAV3['SpeciesName'].isna(),
                                               'Unidentified Halophila',
                                               np.where(SAV3['SpeciesGroup1'] == 'Seagrass', SAV3['CommonIdentifier'],
                                                        SAV3['Drift_Attached'])))

SAV3['analysisunit'] = np.where(SAV3['CommonIdentifier'].isin(species_reject), np.nan,
                                np.where(SAV3['CommonIdentifier'].str.contains('Halophila'), 'Halophila spp.',
                                         np.where(SAV3['SpeciesGroup1'] == 'Seagrass', SAV3['CommonIdentifier'],
                                                  SAV3['Drift_Attached'])))

# Update analysisunit_halid and analysisunit columns for records with Drift_Attached not NA
SAV3.loc[~SAV3['Drift_Attached'].isna(), 'analysisunit_halid'] = SAV3.loc[~SAV3['Drift_Attached'].isna(), 'analysisunit_halid'] + ' algae'
SAV3.loc[~SAV3['Drift_Attached'].isna(), 'analysisunit'] = SAV3.loc[~SAV3['Drift_Attached'].isna(), 'analysisunit'] + ' algae'

In [18]:
# Remove rows with NA in analysisunit column
SAV4 = SAV3.dropna(subset=['analysisunit']).copy()

# Save SAV4 as RDS file / pickle
SAV4.to_pickle("SAV/output/SAV_DataUsed.pkl")

# Save SAV4 as CSV file
SAV4.to_csv("SAV/output/SAV_DataUsed.txt", sep='|', index=False)

In [24]:
# Calculate SAV4_sum by grouping method and ManagedAreaName
SAV4_sum = SAV4.groupby(['method', 'ManagedAreaName']).agg(n_yr=('Year', lambda x: len(np.unique(x))),
                                                          yrs=('Year', lambda x: sorted(np.unique(x)))).reset_index()

### PLOTTING ###

In [30]:
import matplotlib.pyplot as plt

def addfits_blacktrendlines(models, plot_i, param):
    aucol = plot_i.data.columns[0]
    if len(models) == 1:
        i = plot_i.data.iloc[0]['ManagedAreaName']
        filtered_data = SAV4[(SAV4['ManagedAreaName'] == i) & (~SAV4[param].isna()) & (SAV4[aucol] == models[0].data[aucol].unique())]
        plot_i += plt.geom_line(data=filtered_data,
                                mapping=plt.aes(x='relyear', y=models[0].predict(level=0)),
                                color='#000099', size=0.75, alpha=0.7, inherit_aes=False)
        return plot_i

In [31]:
EDA = "plots"
Analyses = ['BB_pct','PA']

In [32]:
# Empty DataFrame to store names of any failed models
failedmods = pd.DataFrame(columns=["model", "error"])

# Create a table of the proportion of present SAV types by managed area and year
props_halid = (SAV4[~SAV4['analysisunit_halid'].str.contains(r'Total|Drift|spp\.', na=False) & ~SAV4['PA'].isna()]
              .groupby(['ManagedAreaName', 'analysisunit_halid', 'relyear'])
              .agg(n_P=('PA', 'sum'), ntot_PA=('PA', 'count'))
              .assign(prop_P=lambda x: x['n_P'] / x['ntot_PA']))

props = (SAV4[~SAV4['analysisunit'].str.contains(r'Total|Drift|decipiens|engelmannii|johnsonii|Unidentified', na=False) & ~SAV4['PA'].isna()]
         .groupby(['ManagedAreaName', 'analysisunit', 'relyear'])
         .agg(n_P=('PA', 'sum'), ntot_PA=('PA', 'count'))
         .assign(prop_P=lambda x: x['n_P'] / x['ntot_PA']))

props_halid.reset_index(inplace=True)
props.reset_index(inplace=True)

props_halid['n_allsp_P'] = props_halid.groupby(['ManagedAreaName', 'relyear'])['n_P'].transform('sum')
props_halid['sp_prop'] = props_halid['n_P'] / props_halid['n_allsp_P']
props_halid['sp_pct'] = props_halid['sp_prop'] * 100

props['n_allsp_P'] = props.groupby(['ManagedAreaName', 'relyear'])['n_P'].transform('sum')
props['sp_prop'] = props['n_P'] / props['n_allsp_P']
props['sp_pct'] = props['sp_prop'] * 100

props2 = pd.concat([props_halid, props]).drop_duplicates()
props2.sort_values(by=['ManagedAreaName', 'relyear', 'analysisunit'], inplace=True)
props = props2

In [33]:
spcollist=["#005396","#0088B1","#00ADAE","#65CCB3","#AEE4C1","#FDEBA8","#F8CD6D","#F5A800","#F17B00"]

In [34]:
spp = ["Halodule wrightii", "Halophila decipiens", "Halophila engelmannii", "Halophila johnsonii", 
         "Halophila spp.", "Ruppia maritima", "Syringodium filiforme", "Thalassia testudinum", "Attached algae"]

spp_common = ["Halophila spp.", "Johnson's seagrass", "Manatee grass", "Paddle grass", 
                "Shoal grass", "Star grass", "Turtle grass", "Widgeon grass", "Attached algae"]

In [35]:
usenames = "common"  # alternative is "scientific"
if usenames == "common":
    spcols = dict(zip(spp_common, spcollist))
else:
    spcols = dict(zip(spp, spcollist))

In [36]:
spindet_nm = ["Unidentified Halophila"]
spindet_cl = spcols["Halophila spp."]

'#005396'

In [37]:
spindet = dict(zip(spindet_nm, [spindet_cl]))

{'Unidentified Halophila': '#005396'}

In [38]:
spcols_list = list(spcols.items())
spcols_list.append(spindet)

[('Halophila spp.', '#005396'),
 ("Johnson's seagrass", '#0088B1'),
 ('Manatee grass', '#00ADAE'),
 ('Paddle grass', '#65CCB3'),
 ('Shoal grass', '#AEE4C1'),
 ('Star grass', '#FDEBA8'),
 ('Turtle grass', '#F8CD6D'),
 ('Widgeon grass', '#F5A800'),
 ('Attached algae', '#F17B00'),
 {'Unidentified Halophila': '#005396'}]

In [39]:
if usenames == "common":
    SAV4.loc[:, 'analysisunit_halid'] = (SAV4['analysisunit_halid'].replace({
        "Thalassia testudinum": "Turtle grass",
        "Syringodium filiforme": "Manatee grass",
        "Halodule wrightii": "Shoal grass",
        "Ruppia maritima": "Widgeon grass",
        "Halophila decipiens": "Paddle grass",
        "Halophila engelmannii": "Star grass",
        "Halophila johnsonii": "Johnson's seagrass",
        "Unidentified Halophila": "Unidentified Halophila",
        "Halophila spp.": "Halophila spp.",
        "Total seagrass": "Total seagrass",
        "Attached algae": "Attached algae",
        "Drift algae": "Drift algae",
        "Total SAV": "Total SAV"
    }))
    
    SAV4.loc[:, 'analysisunit'] = (SAV4['analysisunit'].replace({
        "Thalassia testudinum": "Turtle grass",
        "Syringodium filiforme": "Manatee grass",
        "Halodule wrightii": "Shoal grass",
        "Ruppia maritima": "Widgeon grass",
        "Halophila decipiens": "Paddle grass",
        "Halophila engelmannii": "Star grass",
        "Halophila johnsonii": "Johnson's seagrass",
        "Unidentified Halophila": "Unidentified Halophila",
        "Halophila spp.": "Halophila spp.",
        "Total seagrass": "Total seagrass",
        "Attached algae": "Attached algae",
        "Drift algae": "Drift algae",
        "Total SAV": "Total SAV"
    }))
    
    props.loc[:, 'analysisunit'] = (props['analysisunit'].replace({
        "Thalassia testudinum": "Turtle grass",
        "Syringodium filiforme": "Manatee grass",
        "Halodule wrightii": "Shoal grass",
        "Ruppia maritima": "Widgeon grass",
        "Halophila decipiens": "Paddle grass",
        "Halophila engelmannii": "Star grass",
        "Halophila johnsonii": "Johnson's seagrass",
        "Unidentified Halophila": "Unidentified Halophila",
        "Halophila spp.": "Halophila spp.",
        "Attached algae": "Attached algae"
    }))
    
    props['analysisunit'] = pd.Categorical(props['analysisunit'], categories=[
        "Unidentified Halophila",
        "Halophila spp.",
        "Johnson's seagrass",
        "Manatee grass",
        "Paddle grass",
        "Shoal grass",
        "Star grass",
        "Turtle grass",
        "Widgeon grass",
        "Attached algae"
    ], ordered=True)

In [40]:
import seaborn as sns

prcollist_a = sns.color_palette("YlOrRd", n_colors=len(SAV4['ProgramName'].unique()))
prcollist_b = sns.color_palette("YlGnBu", n_colors=len(SAV4['ProgramName'].unique()) + 1)[::-1]
prcollist = [color for i, color in enumerate(prcollist_a) if i % 2 == 1] + [color for i, color in enumerate(prcollist_b) if i % 2 == 0]
prcollist = prcollist[::-1]

np.random.seed(4691)
progs = np.random.choice(np.sort(SAV4['ProgramName'].unique()), size=len(SAV4['ProgramName'].unique()), replace=False)
prcols = dict(zip(progs, prcollist))

In [41]:
parameters = pd.DataFrame({
    'column': ['BB_all', 'BB_pct', 'PC', 'PO', 'PA'],
    'name': ['Braun Blanquet score', 'Median percent cover', 'Visual percent cover', 'Percent occurrence', 'Frequency of occurrence'],
    'type': ['BBall', 'BBpct', 'PC', 'PO', 'PA']
})
parameters

,column,name,type
0,BB_all,Braun Blanquet score,BBall
1,BB_pct,Median percent cover,BBpct
2,PC,Visual percent cover,PC
3,PO,Percent occurrence,PO
4,PA,Frequency of occurrence,PA


In [42]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plot_theme = plt.rcParams.copy()
plot_theme.update({
    'axes.grid': False,
    'font.family': 'Arial',
    'axes.titlesize': 12,
    'axes.titleweight': 'bold',
    'axes.labelsize': 10,
    'legend.title_fontsize': 10,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'xtick.major.pad': 5,
    'ytick.major.pad': 0,
    'xtick.alignment': 'center'
})

plt.rcParams.update(plot_theme)

In [43]:
ma_halspp = ["Banana River", "Indian River-Malabar to Vero Beach", "Indian River-Vero Beach to Ft. Pierce", "Jensen Beach to Jupiter Inlet",
             "Loxahatchee River-Lake Worth Creek", "Mosquito Lagoon", "Biscayne Bay", "Florida Keys NMS"]

In [44]:
SAV4.ProgramID = SAV4.ProgramID.astype(str)

In [45]:
stats_pct = SAV4[SAV4['ManagedAreaName'].isin(ma_halspp)].groupby(['ManagedAreaName', 'analysisunit']).agg(
    ParameterName=("ParameterName", lambda x: "Median percent cover (from BB scores)"),
    N_Programs=("ProgramID", lambda x: len(x.unique())),
    Programs=("ProgramName", lambda x: ', '.join(sorted(x.unique()))),
    ProgramIDs=("ProgramID", lambda x: ', '.join(sorted(x.unique()))),
    N_Data=("BB_pct", lambda x: len(x[~x.isna()])),
    N_Years=("Year", lambda x: len(x[~x.isna() & ~SAV4['BB_pct'].isna()].unique())),
    EarliestYear=("Year", lambda x: x[~SAV4['BB_pct'].isna()].min()),
    LatestYear=("Year", lambda x: x[~SAV4['BB_pct'].isna()].max()),
    SufficientData=("BB_pct", lambda x: (len(x[~x.isna()]) > 0) & (len(x[~SAV4['BB_pct'].isna()].unique()) >= 5))
).reset_index()

In [117]:
stats_pct2 = SAV4[SAV4['ManagedAreaName'].isin(set(SAV4['ManagedAreaName']).difference(ma_halspp))].groupby(['ManagedAreaName', 'analysisunit_halid']).agg(
    ParameterName=("ParameterName", lambda x: "Median percent cover (from BB scores)"),
    N_Programs=("ProgramID", lambda x: len(x.unique())),
    Programs=("ProgramName", lambda x: ', '.join(sorted(x.unique()))),
    ProgramIDs=("ProgramID", lambda x: ', '.join(sorted(x.unique()))),
    N_Data=("BB_pct", lambda x: len(x[~x.isna()])),
    N_Years=("Year", lambda x: len(x[~x.isna() & ~SAV4['BB_pct'].isna()].unique())),
    EarliestYear=("Year", lambda x: x[~SAV4['BB_pct'].isna()].min()),
    LatestYear=("Year", lambda x: x[~SAV4['BB_pct'].isna()].max()),
    SufficientData=("BB_pct", lambda x: (len(x[~x.isna()]) > 0) & (len(x[~SAV4['BB_pct'].isna()].unique()) >= 5))
).reset_index()

In [124]:
stats_pct2 = stats_pct2.rename(columns={"analysisunit_halid": "analysisunit"})

In [129]:
new_stats_pct = pd.concat([stats_pct, stats_pct2])

In [131]:
new_stats_pct.to_csv("SAV/output/SAV_BBpct_Stats.txt", sep="|", index=False)

In [132]:
stats_pa = SAV4[SAV4['ManagedAreaName'].isin(ma_halspp)].groupby(['ManagedAreaName', 'analysisunit']).agg(
    ParameterName=("ParameterName", lambda x: "Frequency of occurrence"),
    N_Programs=("ProgramID", lambda x: len(x.unique())),
    Programs=("ProgramName", lambda x: ', '.join(sorted(x.unique()))),
    ProgramIDs=("ProgramID", lambda x: ', '.join(map(str, sorted(x.unique())))),
    N_Data=("PA", lambda x: len(x.dropna())),
    N_Years=("Year", lambda x: len(x[~x.isna() & ~x.isna()])),
    EarliestYear=("Year", lambda x: min(x.dropna())),
    LatestYear=("Year", lambda x: max(x.dropna())),
    SufficientData=("Year", lambda x: len(x.dropna()) > 0 and len(x[~x.isna() & ~x.isna()]) >= 5)
).reset_index()

In [133]:
stats_pa2 = SAV4[SAV4['ManagedAreaName'].isin(set(SAV4['ManagedAreaName']).difference(ma_halspp))].groupby(['ManagedAreaName', 'analysisunit_halid']).agg(
    ParameterName=("ParameterName", lambda x: "Frequency of occurrence"),
    N_Programs=("ProgramID", lambda x: len(x.unique())),
    Programs=("ProgramName", lambda x: ', '.join(sorted(x.unique()))),
    ProgramIDs=("ProgramID", lambda x: ', '.join(map(str, sorted(x.unique())))),
    N_Data=("PA", lambda x: len(x.dropna())),
    N_Years=("Year", lambda x: len(x[~x.isna() & ~x.isna()])),
    EarliestYear=("Year", lambda x: min(x.dropna())),
    LatestYear=("Year", lambda x: max(x.dropna())),
    SufficientData=("Year", lambda x: len(x.dropna()) > 0 and len(x[~x.isna() & ~x.isna()]) >= 5)
).reset_index()

In [134]:
stats_pa2['analysisunit'] = stats_pa2['analysisunit_halid']
stats_pa = pd.concat([stats_pa, stats_pa2], ignore_index=True)

stats_pa.loc[stats_pa['N_Years'] == 0, ['EarliestYear', 'LatestYear']] = np.nan

In [135]:
stats_pa

,ManagedAreaName,analysisunit,ParameterName,N_Programs,Programs,ProgramIDs,N_Data,N_Years,EarliestYear,LatestYear,SufficientData,analysisunit_halid
0,Banana River,Drift algae,Frequency of occurrence,1,Seagrass (SJRWMD),3013,5032,5032,1994.0,2022.0,True,NaN
1,Banana River,Halodule wrightii,Frequency of occurrence,1,Seagrass (SJRWMD),3013,9842,9842,1994.0,2022.0,True,NaN
2,Banana River,Halophila spp.,Frequency of occurrence,1,Seagrass (SJRWMD),3013,29597,29597,1994.0,2022.0,True,NaN
3,Banana River,Ruppia maritima,Frequency of occurrence,1,Seagrass (SJRWMD),3013,9849,9849,1994.0,2022.0,True,NaN
4,Banana River,Syringodium filiforme,Frequency of occurrence,1,Seagrass (SJRWMD),3013,9867,9867,1994.0,2022.0,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
187,Terra Ceia,Halodule wrightii,Frequency of occurrence,1,Tampa Bay Seagrass Monitoring,565,627,627,1999.0,2022.0,True,Halodule wrightii
188,Terra Ceia,Ruppia maritima,Frequency of occurrence,1,Tampa Bay Seagrass Monitoring,565,2,2,2016.0,2021.0,False,Ruppia maritima
189,Terra Ceia,Syringodium filiforme,Frequency of occurrence,1,Tampa Bay Seagrass Monitoring,565,492,492,1999.0,2022.0,True,Syringodium filiforme
190,Terra Ceia,Thalassia testudinum,Frequency of occurrence,1,Tampa Bay Seagrass Monitoring,565,1477,1477,1999.0,2022.0,True,Thalassia testudinum


In [38]:
# def rotate_sf(data, x_add=0, y_add=0, ma=None, coast="Atlantic"):
#     def shear_matrix(x):
#         if coast == "Atlantic":
#             if np.unique(ma) in ["Banana River", "Indian River-Malabar to Vero Beach",
#                                  "Indian River-Vero Beach to Ft. Pierce", "Jensen Beach to Jupiter Inlet",
#                                  "Mosquito Lagoon"]:
#                 return np.array([[1, 1.2], [0, 1]])
#             else:
#                 return np.array([[2, 1.2], [0, 1]])
#         else:
#             return np.array([[2, -1.2], [0, 1]])
    
#     def rotate_matrix(x):
#         return np.array([[np.cos(x), np.sin(x)], [-np.sin(x), np.cos(x)]])
    
#     if coast == "Atlantic":
#         if np.unique(ma) in ["Banana River", "Indian River-Malabar to Vero Beach",
#                              "Indian River-Vero Beach to Ft. Pierce", "Jensen Beach to Jupiter Inlet",
#                              "Mosquito Lagoon"]:
#             data['geometry'] = data['geometry'] * shear_matrix(0) * rotate_matrix(np.pi * 0.2) + [x_add, y_add]
#         else:
#             data['geometry'] = data['geometry'] * shear_matrix(0) * rotate_matrix(np.pi / 20) + [x_add, y_add]
#     else:
#         data['geometry'] = data['geometry'] * shear_matrix(0) * rotate_matrix(np.pi * 1.98) + [x_add, y_add]
    
#     return data

In [39]:
# import geopandas as gpd

# # Create model objects, tables, and plots for all MAs with >5 yrs of data
# # Load geospatial data
# GeoDBdate = "6june2023"
# locs_pts = gpd.read_file(f"SAV/mapping/SampleLocations{GeoDBdate}/seacar_dbo_vw_SampleLocation_Point.shp")
# locs_lns = gpd.read_file(f"SAV/mapping/SampleLocations{GeoDBdate}/seacar_dbo_vw_SampleLocation_Line.shp")
# rcp = gpd.read_file("SAV/mapping/orcp_all_sites/orcp_all_sites/ORCP_Managed_Areas.shp")
# counties = gpd.read_file("SAV/mapping/FLCounties/Counties_-_Detailed_Shoreline.shp")
# corners = pd.read_csv("SAV/mapping/MApolygons_corners.csv")

# # Add 20% of difference (xmax-xmin) to xmax to help prevent year labels from getting cut off map images and 10% to ymax
# corners["xmax"] = corners["xmax"] + (corners["xmax"] - corners["xmin"]) * 0.25
# corners["ymax"] = corners["ymax"] + (corners["ymax"] - corners["ymin"]) * 0.1

# # Make geometries valid
# locs_pts = make_valid(locs_pts)
# locs_lns = make_valid(locs_lns)
# rcp = make_valid(rcp)
# counties = make_valid(counties)

# # Transform geometries to CRS 4326
# locs_pts = locs_pts.to_crs(4326)
# locs_lns = locs_lns.to_crs(4326)
# rcp = rcp.to_crs(4326)
# counties = counties.to_crs(4326)

# # Perform spatial intersection
# locs_pts_rcp = gpd.sjoin(locs_pts, rcp, op="intersects")
# locs_lns_rcp = gpd.sjoin(locs_lns, rcp, op="intersects")

# # Merge with pnames
# pnames = SAV4.groupby(["ProgramID", "ProgramName"]).size().reset_index().rename(columns={0: "Count"})
# locs_pts_rcp = locs_pts_rcp.merge(pnames, on="ProgramID", how="left")
# locs_lns_rcp = locs_lns_rcp.merge(pnames, on="ProgramID", how="left")